# Neccesary Libraries

In [ ]:
%pip install pandas
%pip install python-dotenv
%pip install matplotlib

In [ ]:
import ast
import json
import matplotlib.pyplot as plt
import os
import pandas as pd
import plotly.express as px
import seaborn as sns

# Parameters

In [ ]:
def extract_themes(csv_path):
    df = pd.read_csv(csv_path, sep=';', header=None)
    themes = []
    for col in df.columns:
        for val in df[col].dropna():
            line = str(val).strip()
            if line.startswith('===') and line.endswith('==='):
                theme_name = line.strip('=').strip()
                themes.append(theme_name)
    return themes

In [ ]:
# Themes loader (change to the csv you want)
csv_path = 'themes/Criterios_extendidos.csv'
themes = extract_themes(csv_path)
print(themes)

# Classifier

In [ ]:
def transform_forum_results(input_path, output_path):
    df = pd.read_csv(input_path)
    rows = []
    for _, row in df.iterrows():
        try:
            matched = ast.literal_eval(str(row.get('matched_themes', '{}')))
        except Exception:
            matched = {}
        for theme, keywords in matched.items():
            rows.append({
                'theme': theme,
                'title': row.get('title', ''),
                'url': row.get('url', ''),
                'keywords': ', '.join(keywords) if isinstance(keywords, list) else str(keywords),
                'original_keywords': ', '.join(keywords) if isinstance(keywords, list) else str(keywords),
                'original_theme': theme,
                'publish_date': row.get('publish_date', '')
            })
    out_df = pd.DataFrame(rows)
    out_df.to_csv(output_path, index=False)

def transform_news_results(input_path, output_path):
    df = pd.read_csv(input_path)
    columns = ['theme', 'title', 'url', 'keywords', 'original_keywords', 'original_theme', 'publish_date']
    if set(columns).issubset(df.columns):
        df = df[columns]
    else:
        df['original_keywords'] = df['keywords']
        df['original_theme'] = df['theme']
        df = df[columns]
    df.to_csv(output_path, index=False)

def transform_theme_analysis_json_to_csv(json_path, output_path):
    # Lee el archivo JSON de results y lo transforma al formato de theme_analysis de forums/news
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    theme_analysis = data.get('theme_analysis', {})
    rows = []
    for theme, info in theme_analysis.items():
        # Para forum_analysis_report.json: info tiene 'posts' y 'count'
        article_count = info.get('count', 0) or info.get('articles_count', 0)
        # Extraer keywords únicos de los posts/artículos
        keywords_set = set()
        posts = info.get('posts', []) or info.get('articles', [])
        for post in posts:
            # Puede haber 'keywords_found' o no
            if 'keywords_found' in post:
                kws = post['keywords_found']
                if isinstance(kws, list):
                    keywords_set.update(kws)
                elif isinstance(kws, str):
                    keywords_set.add(kws)
        rows.append({
            'theme': theme,
            'article_count': article_count,
            'unique_keywords_found': len(keywords_set),
            'percentage_of_total': None  # Se puede calcular después si se desea
        })
    df = pd.DataFrame(rows)
    # Calcular porcentaje del total
    total = df['article_count'].sum()
    if total > 0:
        df['percentage_of_total'] = df['article_count'] / total * 100
    df.to_csv(output_path, index=False)

# Rutas de entrada y salida
results_dir = 'results'
forums_dir = 'forums'
news_dir = 'news'

# Transformar forum_analysis_posts.csv → forums_full_with_dates.csv
try:
    os.makedirs(forums_dir, exist_ok=True)
    transform_forum_results(
        os.path.join(results_dir, 'forum_analysis_posts.csv'),
        os.path.join(forums_dir, 'forums_full_with_dates.csv')
    )
    print("Transformación de forum_analysis_posts.csv completada.")
except Exception as e:
    print(f"Error transforming forum results: {e}")

# Transformar ai_news_analysis_articles.csv → reclassified_analysis_full_with_dates2.csv
try:
    os.makedirs(news_dir, exist_ok=True)
    transform_news_results(
        os.path.join(results_dir, 'ai_news_analysis_articles.csv'),
        os.path.join(news_dir, 'reclassified_analysis_full_with_dates2.csv')
    )
    print("Transformación de ai_news_analysis_articles.csv completada.")
except Exception as e:
    print(f"Error transforming news results: {e}")

# Transformar forum_analysis_report.json → forums_reclassified_theme_analysis.csv
try:
    os.makedirs(forums_dir, exist_ok=True)
    transform_theme_analysis_json_to_csv(
        os.path.join(results_dir, 'forum_analysis_report.json'),
        os.path.join(forums_dir, 'forums_reclassified_theme_analysis.csv')
    )
    print("Transformación de forum_analysis_report.json completada.")
except Exception as e:
    print(f"Error transforming forum theme analysis: {e}")

# Transformar ai_news_analysis_report.json → reclassified_analysis_theme_analysis.csv
try:
    os.makedirs(news_dir, exist_ok=True)
    transform_theme_analysis_json_to_csv(
        os.path.join(results_dir, 'ai_news_analysis_report.json'),
        os.path.join(news_dir, 'reclassified_analysis_theme_analysis.csv')
    )
    print("Transformación de ai_news_analysis_report.json completada.")
except Exception as e:
    print(f"Error transforming news theme analysis: {e}")

# Eda Noticias

In [ ]:
class NewsVisualizer:
    def __init__(self, full_file, theme_analysis_file):
        """
        Inicializa el visualizador con los archivos de datos
        """
        self.full_df = pd.read_csv(full_file)
        self.theme_analysis = pd.read_csv(theme_analysis_file)


        plt.style.use('ggplot')

        self.output_dir = Path('visualizations-news')
        self.output_dir.mkdir(exist_ok=True)

    def _prepare_date_data(self):
        """Prepara los datos para la visualización temporal"""
        # Convertir la columna de publish_date a datetime
        self.full_df['year'] = pd.to_datetime(self.full_df['publish_date']).dt.year

        # Agrupar por tema y año
        temporal_data = self.full_df.groupby(['theme', 'year']).size().reset_index(name='count')
        return temporal_data

    def create_theme_distribution_bar(self):
        """
        Crea un gráfico de barras horizontal con la distribución de artículos por tema
        """
        # Ordenar temas por cantidad de artículos
        theme_analysis_sorted = self.theme_analysis.sort_values('article_count', ascending=True)

        # Crear figura con tamaño específico
        plt.figure(figsize=(12, 10))

        # Crear gráfico de barras horizontal
        bars = plt.barh(theme_analysis_sorted['theme'],
                       theme_analysis_sorted['article_count'],
                       color=sns.color_palette("husl", len(theme_analysis_sorted)))

        # Agregar etiquetas en las barras
        for bar in bars:
            width = bar.get_width()
            plt.text(width, bar.get_y() + bar.get_height()/2,
                    f'{int(width)}',
                    va='center', ha='left', fontsize=10)

        # Ajustar layout y etiquetas
        plt.title('Distribución de Artículos por Tema', pad=20, fontsize=14)
        plt.xlabel('Número de Artículos', fontsize=12)
        plt.ylabel('Tema', fontsize=12)

        plt.tight_layout()

        # Guardar figura
        plt.savefig(self.output_dir / 'theme_distribution.png',
                   dpi=300,
                   bbox_inches='tight')
        plt.close()

    def create_theme_percentage_pie(self):
        """
        Crea un gráfico circular con los porcentajes por tema
        """
        # Ordenar por porcentaje
        theme_analysis_sorted = self.theme_analysis.sort_values('percentage_of_total', ascending=False)

        # Crear figura
        plt.figure(figsize=(12, 8))

        # Crear gráfico circular
        plt.pie(theme_analysis_sorted['percentage_of_total'],
                labels=theme_analysis_sorted['theme'],
                autopct='%1.1f%%',
                colors=sns.color_palette("husl", len(theme_analysis_sorted)),
                wedgeprops={'edgecolor': 'white'},
                textprops={'fontsize': 8})

        plt.title('Distribución Porcentual de Artículos por Tema', pad=20, fontsize=14)

        plt.tight_layout()

        plt.savefig(self.output_dir / 'theme_percentage.png',
                   dpi=300,
                   bbox_inches='tight')
        plt.close()

    def create_temporal_bubble_chart(self):
        """
        Crea un bubble chart interactivo usando Plotly con mejores dimensiones y formato
        """
        # Preparar datos
        temporal_data = self._prepare_date_data()

        # Crear diccionario de nombres cortos para los temas
        theme_shortcuts = global_shortcuts

        # Aplicar nombres cortos
        temporal_data['theme_short'] = temporal_data['theme'].map(theme_shortcuts)

        # Importar plotly
        import plotly.express as px

        # Crear bubble chart
        fig = px.scatter(
            temporal_data,
            x='year',
            y='theme_short',  # Usar nombres cortos
            size='count',
            color='theme_short',  # Usar nombres cortos para colores
            title='Distribución de artículos por año y tema',
            labels={
                'count': 'Cantidad de artículos',
                'year': 'Año',
                'theme_short': 'Tema'  # Actualizar etiqueta
            },
            size_max=60,  # Aumentar tamaño máximo de las burbujas
            hover_data={
                'theme_short': False,  # No mostrar en hover
                'count': True,
                'year': True
            },
            hover_name='theme_short'  # Mostrar tema en el título del hover
        )

        # Ajustar layout
        fig.update_layout(
            height=800,
            width=1200,  # Especificar ancho
            showlegend=True,
            title_x=0.5,
            title_y=0.95,
            margin=dict(l=20, r=20, t=80, b=20),  # Ajustar márgenes
            xaxis=dict(
                tickmode='linear',
                tick0=temporal_data['year'].min(),
                dtick=1,
                range=[
                    temporal_data['year'].min()-0.5,
                    temporal_data['year'].max()+0.5
                ]
            ),
            yaxis=dict(
                title_font=dict(size=14),  # Aumentar tamaño de fuente
                tickfont=dict(size=12)     # Aumentar tamaño de fuente
            ),
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=1.02,
                font=dict(size=12)
            ),
            hoverlabel=dict(
                bgcolor="white",
                font_size=14
            )
        )

        # Guardar como HTML
        fig.write_html(
            str(self.output_dir / 'temporal_bubble_chart.html'),
            include_plotlyjs='cdn',  # Usar CDN para plotly.js
            full_html=True
        )

    def create_theme_evolution_line(self):
        """
        Crea un gráfico de líneas que muestra la evolución temporal de cada tema
        """
        temporal_data = self._prepare_date_data()

        # Crear figura
        plt.figure(figsize=(15, 8))

        # Crear líneas para cada tema
        for theme in temporal_data['theme'].unique():
            theme_data = temporal_data[temporal_data['theme'] == theme]
            plt.plot(theme_data['year'],
                    theme_data['count'],
                    marker='o',
                    linewidth=2,
                    label=theme)

        # Ajustar ejes y etiquetas
        plt.title('Evolución Temporal del Número de Artículos por Tema', pad=20, fontsize=14)
        plt.xlabel('Año', fontsize=12)
        plt.ylabel('Número de Artículos', fontsize=12)

        # Rotar etiquetas del eje x
        plt.xticks(rotation=45)

        # Agregar leyenda
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

        # Agregar grid
        plt.grid(True, linestyle='--', alpha=0.7)

        # Ajustar layout
        plt.tight_layout()

        # Guardar figura
        plt.savefig(self.output_dir / 'theme_evolution.png',
                   dpi=300,
                   bbox_inches='tight')
        plt.close()

    def create_all_visualizations(self):
        """
        Crea todas las visualizaciones
        """
        print("Creando visualizaciones...")

        print("1. Generando gráfico de distribución por tema...")
        self.create_theme_distribution_bar()

        print("2. Generando gráfico de porcentajes...")
        self.create_theme_percentage_pie()

        print("3. Generando gráfico de distribución temporal...")
        self.create_temporal_bubble_chart()

        print("4. Generando gráfico de evolución temporal...")
        self.create_theme_evolution_line()

        print(f"\nVisualizaciones guardadas en el directorio: {self.output_dir}")
        print("Archivos generados:")
        print("- theme_distribution.png: Gráfico de barras con la distribución por tema")
        print("- theme_percentage.png: Gráfico circular con porcentajes")
        print("- temporal_distribution.png: Gráfico de distribución temporal")
        print("- theme_evolution.png: Gráfico de evolución temporal por tema")

def main():
    # Inicializar visualizador
    try:
        visualizer = NewsVisualizer(
            'news/reclassified_analysis_full_with_dates2.csv',
            'news/reclassified_analysis_theme_analysis.csv'
        )
    except FileNotFoundError as e:
        print(f"Error: {e}. Asegúrate de que los archivos existen y están en el formato correcto.")
        return
    except Exception as e:
        print(f"Error: {e}")
        return

    # Generar todas las visualizaciones
    visualizer.create_all_visualizations()

if __name__ == "__main__":
    main()

# Eda Foros

In [ ]:
class RedditVisualizer:
    def __init__(self, full_file, theme_analysis_file):
        self.full_df = pd.read_csv(full_file)
        self.theme_analysis = pd.read_csv(theme_analysis_file)
        self.output_dir = Path('visualizations-forums')
        self.output_dir.mkdir(exist_ok=True)

        self.theme_shortcuts = global_shortcuts

    def create_distribution_bar(self):
        """Bar chart de distribución por tema"""
        # Asegurarse de que los datos están correctamente formateados
        data = (self.full_df.groupby('theme')
               .size()
               .reset_index(name='count')
               .sort_values('count', ascending=True))

        # Aplicar nombres cortos
        data['theme_short'] = data['theme'].map(self.theme_shortcuts)

        plt.figure(figsize=(12, 8))
        bars = plt.barh(data['theme_short'],
                       data['count'],
                       color=sns.color_palette("husl", len(data)))

        # Añadir etiquetas
        for i, bar in enumerate(bars):
            width = bar.get_width()
            plt.text(width, i, f' {int(width)}',
                    va='center', ha='left')

        plt.title('Distribución de Artículos por Tema', pad=20, fontsize=14)
        plt.xlabel('Número de Artículos', fontsize=12)
        plt.ylabel('Tema', fontsize=12)

        plt.tight_layout()
        plt.savefig(self.output_dir / 'theme_distribution.png',
                   dpi=300, bbox_inches='tight')
        plt.close()

    def create_percentage_pie(self):
        """Pie chart de porcentajes"""
        # Calcular porcentajes
        data = (self.full_df.groupby('theme')
               .size()
               .reset_index(name='count'))
        total = data['count'].sum()
        data['percentage'] = (data['count'] / total) * 100

        # Aplicar nombres cortos
        data['theme_short'] = data['theme'].map(self.theme_shortcuts)

        # Ordenar por porcentaje
        data = data.sort_values('percentage', ascending=False)

        plt.figure(figsize=(12, 8))
        wedges, texts, autotexts = plt.pie(data['percentage'],
                                         labels=data['theme_short'],
                                         autopct='%1.1f%%',
                                         colors=sns.color_palette("husl", len(data)))

        # Ajustar propiedades del texto
        plt.setp(autotexts, size=8, weight="bold")
        plt.setp(texts, size=8)

        plt.title('Distribución Porcentual de Artículos por Tema',
                 pad=20, fontsize=14)

        plt.tight_layout()
        plt.savefig(self.output_dir / 'theme_percentage.png',
                   dpi=300, bbox_inches='tight')
        plt.close()

    def create_bubble_chart(self):
        """Bubble chart interactivo con Plotly"""
        # Preparar datos temporales
        self.full_df['year'] = pd.to_datetime(self.full_df['publish_date']).dt.year
        temporal_data = (self.full_df.groupby(['year', 'theme'])
                        .size()
                        .reset_index(name='count'))

        # Aplicar nombres cortos
        temporal_data['theme_short'] = temporal_data['theme'].map(self.theme_shortcuts)

        fig = px.scatter(temporal_data,
                        x='year',
                        y='theme_short',
                        size='count',
                        color='theme_short',
                        title='Distribución de artículos por año y tema',
                        labels={'count': 'Cantidad de artículos',
                               'year': 'Año',
                               'theme_short': 'Tema'},
                        size_max=50)

        fig.update_layout(
            height=800,
            showlegend=True,
            title_x=0.5,
            xaxis=dict(
                tickmode='linear',
                tick0=temporal_data['year'].min(),
                dtick=1,
                range=[temporal_data['year'].min()-0.5,
                       temporal_data['year'].max()+0.5]
            )
        )

        fig.write_html(str(self.output_dir / 'temporal_bubble_chart.html'))

    def create_all_visualizations(self):
        print("Creando visualizaciones...")
        self.create_distribution_bar()
        print("✓ Gráfico de distribución creado")
        self.create_percentage_pie()
        print("✓ Gráfico de porcentajes creado")
        self.create_bubble_chart()
        print("✓ Bubble chart creado")
        print(f"\nVisualizaciones guardadas en: {self.output_dir}")

def main():
    try:
        visualizer = RedditVisualizer(
            'forums/forums_full_with_dates.csv',
            'forums/forums_reclassified_theme_analysis.csv'
        )
        visualizer.create_all_visualizations()
    except FileNotFoundError as e:
        print(f"Error: {e}. Asegúrate de que los archivos existen y están en el formato correcto.")
        return
    except Exception as e:
        print(f"Error: {e}")
        return

if __name__ == "__main__":
    main()

# Cargar el dataframe antes de generar las visualizaciones sankey

In [ ]:
# import pandas as pd
# import plotly.graph_objects as go
# from collections import defaultdict

# def create_sankey_diagram(df, min_connections=5):  # Añadimos umbral mínimo
#     theme_shortcuts = global_shortcuts

#     # Recolectar conexiones
#     article_themes = defaultdict(list)
#     for _, row in df.iterrows():
#         themes = row['theme'].split(' . ')
#         short_themes = [theme_shortcuts.get(theme, theme) for theme in themes]
#         article_themes[row['url']].extend(short_themes)

#     # Contar conexiones y temas
#     theme_connections = defaultdict(int)
#     theme_counts = defaultdict(int)

#     for themes in article_themes.values():
#         # Contar frecuencia individual
#         for theme in themes:
#             theme_counts[theme] += 1

#         # Crear conexiones
#         if len(themes) > 1:
#             for i in range(len(themes)):
#                 for j in range(i + 1, len(themes)):
#                     theme1, theme2 = sorted([themes[i], themes[j]])
#                     theme_connections[(theme1, theme2)] += 1

#     # Filtrar conexiones por umbral mínimo
#     significant_connections = {k: v for k, v in theme_connections.items() if v >= min_connections}

#     # Obtener temas que tienen conexiones significativas
#     connected_themes = set()
#     for (source, target) in significant_connections.keys():
#         connected_themes.add(source)
#         connected_themes.add(target)

#     # Crear índices solo para temas conectados
#     all_themes = sorted(list(connected_themes))
#     theme_to_index = {theme: idx for idx, theme in enumerate(all_themes)}

#     # Preparar datos para Sankey
#     sources = []
#     targets = []
#     values = []

#     for (source, target), value in significant_connections.items():
#         sources.append(theme_to_index[source])
#         targets.append(theme_to_index[target])
#         values.append(value)

#     # Colores más suaves
#     node_colors = [
#         '#FF9999', '#66B2FF', '#99FF99', '#FFCC99',
#         '#FF99CC', '#99CCFF', '#FFB366', '#99FF99'
#     ]

#     # Crear figura con diseño mejorado
#     fig = go.Figure(data=[go.Sankey(
#         arrangement = "snap",  # Cambiado a "snap" para mejor disposición
#         node = dict(
#             pad = 20,
#             thickness = 20,
#             line = dict(color = "black", width = 0.5),
#             label = all_themes,
#             color = node_colors[:len(all_themes)],
#             customdata = [theme_counts[theme] for theme in all_themes],
#             hovertemplate = 'Tema: %{label}<br>Total de artículos: %{customdata}<extra></extra>'
#         ),
#         link = dict(
#             source = sources,
#             target = targets,
#             value = values,
#             hovertemplate = 'De %{source.label}<br>' +
#                           'A %{target.label}<br>' +
#                           'Artículos compartidos: %{value}<extra></extra>',
#             color = ['rgba(150,150,150,0.3)'] * len(values)  # Conexiones más sutiles
#         )
#     )])

#     fig.update_layout(
#         title = dict(
#             text = "Conexiones entre Temas<br>" +
#                    f"<sup>Mostrando conexiones con {min_connections} o más artículos compartidos</sup>",
#             font = dict(size=20)
#         ),
#         font_size = 12,
#         height = 800,
#         width = 1200,
#         plot_bgcolor = 'white',
#         paper_bgcolor = 'white'
#     )

#     return fig

# # Crear y mostrar el diagrama
# fig = create_sankey_diagram(df, min_connections=10)  # Ajusta este valor según necesites
# fig.show()

# # Guardar como HTML
# fig.write_html("tema_conexiones_sankey.html")